In [1]:
import torchvision.models as models
import torch
import torch.nn as nn
from data_feat_ex2 import ImageBagDataset, transform, make_deterministic_dataloader, mil_collate_fn, set_seed, DenseNet121Extractor
from torch.utils.data import DataLoader, Subset
from torchmil.data import collate_fn
from train_val import train, val
import random
import numpy as np
from sklearn.model_selection import StratifiedKFold

In [2]:
#features_path = "/home/silvia.collicelli/project/embeddings/features"
#labels_path = "/home/silvia.collicelli/project/embeddings/labels"
#root_dir = "/home/silvia.collicelli/data/Dataset"
#annotations_file = "/home/silvia.collicelli/data/controlled_dataset_metadata.parquet"
features_path = "C:\\Users\\utente\\Documents\\UNI\\MAGISTRALE\\tesi\\raw_dataset\\artifacts\\embeddings\\features"
labels_path = "C:\\Users\\utente\\Documents\\UNI\\MAGISTRALE\\tesi\\raw_dataset\\artifacts\\embeddings\\labels"
root_dir = "C:\\Users\\utente\\Documents\\UNI\\MAGISTRALE\\tesi\\raw_dataset\\artifacts\\Dataset"
annotations_file = "C:\\Users\\utente\\Documents\\UNI\\MAGISTRALE\\tesi\\raw_dataset\\artifacts\\clinical_case_metadata.parquet"

k_folds=5
seed=0
set_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
dataset = ImageBagDataset(root_dir, annotations_file, transform)
feat_ex = DenseNet121Extractor(True)

In [5]:
feat_ex(dataset[0]['X']).size()

torch.Size([4, 1024])

In [ ]:
train_loader = make_deterministic_dataloader(
    dataset=dataset,
    batch_size=2,
    num_workers=2,
    pin_memory=False,
    shuffle=True,
    offset=0,
    base_seed=0,
    sampler=None,
    collate_fn=mil_collate_fn,
)

In [ ]:
model = End2EndABMIL(False).to(device)
#emb = model(dataset[0]['X'])
#dataset[0]['X'].unsqueeze(0).shape


In [13]:
#emb2 = np.load("C:\\Users\\utente\\Documents\\UNI\\MAGISTRALE\\tesi\\raw_dataset\\artifacts\\embeddings\\features16\\5H6mQZJMmqo9LPcFd1QOW.npy")
#emb2 = torch.tensor(emb2)
#emb2 = emb2.unsqueeze(0)
#print(emb[0][0], emb2[0][0])

In [14]:
#seed=0
#torch.manual_seed(seed)
#random.seed(seed)
#np.random.seed(seed)
#mil = ABMIL((1024,), 256, "relu")
#print(mil(emb), mil(emb2))

In [15]:
cv = StratifiedKFold(k_folds, shuffle=True)
dataset = ImageBagDataset(root_dir, annotations_file, transform)
bag_labels = [dataset[i]["Y"].item() for i in range(len(dataset))]
train_data = []
val_data = []

for _, (train_idx, val_idx) in enumerate(cv.split(np.zeros(len(bag_labels)), bag_labels)):
    train_data.append(Subset(dataset, train_idx))
    val_data.append(Subset(dataset, val_idx))

train_dataloader = DataLoader(
    train_data[0], batch_size=2, shuffle=True, collate_fn=collate_fn
)
val_dataloader = DataLoader(
    val_data[0], batch_size=2, shuffle=False, collate_fn=collate_fn
)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_train = End2EndABMIL(train_last_block=True)
model_freeze = End2EndABMIL(train_last_block=False)
model_train.to(device)
model_freeze.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer_train = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model_train.parameters()),
    lr=1e-4
)
optimizer_freeze = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model_freeze.parameters()),
    lr=1e-4
)
#data[0]

In [ ]:
import torch.nn as nn

def set_frozen_modules_to_eval(model: nn.Module):
    for name, module in model.named_modules():
        # Skip the top-level module itself
        if module is model:
            continue

        params = list(module.parameters(recurse=False))
        if not params:
            continue

        # Check if all parameters in this module are frozen
        if isinstance(module, nn.BatchNorm2d):
            module.eval()
        if all(not p.requires_grad for p in params):
            module.eval()

set_frozen_modules_to_eval(model_freeze)

BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
BatchNorm2d(224, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
BatchNorm2d(128, eps=1e-05, mom

In [17]:
#print("\tTRAINING\t\t\t\t\t\t\tVALIDATION")
for epoch in range(10):     
    #train_losst, train_acct = train(model_train, device, criterion, optimizer_train, train_dataloader)
    train_lossf, train_accf = train(model_freeze, device, criterion, optimizer_freeze, train_dataloader)

    #val_losst, val_acct, stop = val(model_train, device, criterion, val_dataloader, epoch, additional_metrics=False)
    val_lossf, val_accf, stop = val(model_freeze, device, criterion, val_dataloader, epoch, additional_metrics=False)

    #print(f"lt: {train_losst: .4f}\t lf: {train_lossf: .4f}\t at: {train_acct: .4f}\t af {train_accf: .4f}\t||\tlt: {val_losst: .4f}\t lf: {val_lossf: .4f}\t at: {val_acct: .4f}\t af {val_accf: .4f}")
    print(f"Epoch {epoch+1} | Train Loss: {train_lossf: .4f} | Val Loss: {val_lossf: .4f}")

Epoch 1 | Train Loss:  0.7282 | Val Loss:  0.7320
Epoch 2 | Train Loss:  0.7337 | Val Loss:  0.7050
Epoch 3 | Train Loss:  0.7091 | Val Loss:  0.6726
Epoch 4 | Train Loss:  0.6761 | Val Loss:  0.6455
Epoch 5 | Train Loss:  0.6628 | Val Loss:  0.6268
Epoch 6 | Train Loss:  0.6552 | Val Loss:  0.6133
Epoch 7 | Train Loss:  0.6295 | Val Loss:  0.6101
Epoch 8 | Train Loss:  0.6225 | Val Loss:  0.6046
Epoch 9 | Train Loss:  0.6250 | Val Loss:  0.6015
Epoch 10 | Train Loss:  0.5931 | Val Loss:  0.6015


In [18]:
for epoch in range(10):     
    #train_losst, train_acct = train(model_train, device, criterion, optimizer_train, train_dataloader)
    train_lossf, train_accf = train(model_train, device, criterion, optimizer_train, train_dataloader)

    #val_losst, val_acct, stop = val(model_train, device, criterion, val_dataloader, epoch, additional_metrics=False)
    val_lossf, val_accf, stop = val(model_train, device, criterion, val_dataloader, epoch, additional_metrics=False)

    #print(f"lt: {train_losst: .4f}\t lf: {train_lossf: .4f}\t at: {train_acct: .4f}\t af {train_accf: .4f}\t||\tlt: {val_losst: .4f}\t lf: {val_lossf: .4f}\t at: {val_acct: .4f}\t af {val_accf: .4f}")
    print(f"Epoch {epoch+1} | Train Loss: {train_lossf: .4f} | Val Loss: {val_lossf: .4f}")

Epoch 1 | Train Loss:  0.7355 | Val Loss:  0.5645
Epoch 2 | Train Loss:  0.5971 | Val Loss:  0.5471
Epoch 3 | Train Loss:  0.5014 | Val Loss:  0.5474
Epoch 4 | Train Loss:  0.4201 | Val Loss:  0.5423
Epoch 5 | Train Loss:  0.4020 | Val Loss:  0.5577
Epoch 6 | Train Loss:  0.3492 | Val Loss:  0.5572
Epoch 7 | Train Loss:  0.2389 | Val Loss:  0.5682
Epoch 8 | Train Loss:  0.1754 | Val Loss:  0.6433
Epoch 9 | Train Loss:  0.4226 | Val Loss:  0.5891
Epoch 10 | Train Loss:  0.2614 | Val Loss:  0.6341


In [1]:
from data_feat_ex2 import DenseNet121Extractor, ABMIL

In [2]:
feat_ex = DenseNet121Extractor(True)
for name, param in feat_ex.model.features.norm5.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")

weight: requires_grad=True
bias: requires_grad=True


In [4]:
from torchvision import models
model = models.densenet121(
    weights=models.DenseNet121_Weights.DEFAULT
    )
model.features.norm5

BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [ ]:
model = ABMIL(
    device=device,
    in_shape=in_shape,
    att_dim=config.att_dim,
    att_act=config.att_act,
    feat_ext=feat_ex,
    train_backbone=True
    )

_DenseBlock(
  (denselayer1): _DenseLayer(
    (norm1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (conv1): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (denselayer2): _DenseLayer(
    (norm1): BatchNorm2d(544, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (conv1): Conv2d(544, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (denselayer3): _DenseLayer(
    (norm1): BatchNorm2d(576, eps=1e-05, moment